In [ ]:
#import libraries

import pandas as pd
import requests
import credentials
import time

from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
#importing key from separate python file

key = credentials.youtube

In [ ]:
sentiment_analyzer = SentimentIntensityAnalyzer()

In [ ]:
#importing data

path = 'c:/Users/Diego/GitHub/gothamchess_data_stats.csv'
import_df = pd.read_csv(path)

In [ ]:
def get_sentiment_values(comment) : #function to parse comments and return sentiment scores
    sentiment = sentiment_analyzer.polarity_scores(comment)

    pos = sentiment['pos']
    neu = sentiment['neu']
    neg = sentiment['neg']
    comp = sentiment['compound']

    return pos, neu, neg, comp

In [ ]:
def get_sentiment(df) : #function to make api call and iterate over video ids in url and collect video's comments

    for index, row in import_df.iterrows():
        video_id = row['video_id']
        pageToken = ''
        
        while 1:
            url_comments = f'https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults=100&videoId={video_id}&key={key}&order=relevance&{pageToken}'
            comment_api = requests.get(url_comments).json()
        
            time.sleep(1)
            for comments in comment_api['items'] :
                comment = comments['snippet']['topLevelComment']['snippet']['textDisplay']

                pos, neu, neg, comp = get_sentiment_values(comment) #parsing comment to grab sentiment values

                temp_df = pd.DataFrame({
                    'video_id' : [video_id],
                    'positive_value' : [pos],
                    'neutral_value' : [neu],
                    'negative_value' : [neg],
                    'compound' : [comp]})

                df = pd.concat([df, temp_df], ignore_index=True)

            try: 
                if comment_api['nextPageToken'] != None: #as long as 'nextPageToken' is not blank the loop will continue and grab string to be used in next api call
                    pageToken = 'pageToken=' + comment_api['nextPageToken'] 
            except: #once 'nextPageToken' is blank the loop will break
                break

    return df

In [ ]:
df = pd.DataFrame(columns=['video_id','positive_value','neutral_value','negative_value','compound'])

dfn = get_sentiment(df)

In [ ]:
def df_formatting(table) : #function to parse dataframe and apply formatting
    avg = table.groupby(['video_id']).mean().round(2) #grouping values by video and rounding
    join_df = import_df.join(avg, on=['video_id'], how='left') #left join to dataframe that was imported
    join_df = join_df.drop(columns=['Unnamed: 0']) #dropping unnecessary column - will need to fix

    return join_df

In [ ]:
main_frame = df_formatting(dfn)

In [ ]:
main_frame